In [1]:
import sys
sys.path.append("..") # append the directory above where the databese is.
from utils import config
import pandas as pd
import numpy as np
import sqlite3
#import xgboost as xgb
from numba import jit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
%load_ext line_profiler
### disable scientific notation in pandas
pd.set_option('display.float_format', '{:.2f}'.format) ### display up to 2 decimal pts

In [ ]:
# Combine four years in one row and create the new dataframe, still experimental takes aprox 3 minutes
def combine_years(df):
    row_aux = pd.Series()
    row_final = pd.Series()
    dfs = []
    n = 1
    d = {column:column for column in df.columns}
    for index in df.index:
        row_aux = df.loc[index].rename({column: f"{column}.year.{index[1]}" for column in df}) # rename all columns
        row_final = row_final.append(row_aux)
        if np.mod(n, 4) == 0:
            dfs.append(row_final.to_frame().T)
            row_aux = pd.Series()
            row_final = pd.Series()
        n += 1
    return pd.concat(dfs)
%lprun -f  combine_years df_final = combine_years(df_cleaned)

In [2]:
def add_others_GDP(df_origin):
    df = df_origin[config.GDP].copy()
    n_countries = df.index.get_level_values(level="CountryCode").nunique()
    countries = df.index.get_level_values(level="CountryCode").unique()
    n_years = df.index.get_level_values(level="Year").nunique()
    n_rows = df.shape[0]
    dfs = []
    m = 0
    while(m + n_years <= n_rows):
        df_slice = df.iloc[m:m + n_years]
        dfs.append(df_slice)
        m += n_years
    # actually dfs elements are series, we will keep working with them because I've seen it is lighter to work with
    # series than with dataframes
    for i in range(len(dfs)):
        df_aux = dfs[i]
        df_copies = []
        for j in range(n_countries):
            df_copies.append(dfs[i].copy())
        dfs[i] = pd.concat(df_copies)
    i = 0
    for series in dfs:
        series.rename(f"{config.GDP}_{countries[i]}", inplace=True)
        i += 1
    dfs_frames = [df.to_frame().reset_index(drop=True) for df in dfs]
    dfs_joined = dfs_frames[0].join(dfs_frames[1:], how="left")
    dfs_joined.set_index(df.index, inplace=True)
    df_final = df.join(dfs_joined)
    return df_final.drop(config.GDP) # now we don't want the original gdp as it's repeated in the column with key f"config.GDP_{country}"


In [3]:
with sqlite3.connect(config.DATABASE_PATH) as connection:
    df = pd.read_sql("SELECT * FROM CountryIndicators", connection)

In [ ]:
def rm_countries_no_GDP(df):
    countries_gdp = df[df["IndicatorCode"] == config.GDP]["CountryCode"].to_list() #countries with gdp
    countries = df["CountryCode"].unique()
    countries_no_gdp = list(set(sorted(countries)) - set(sorted(countries_gdp))) # set difference =  countries with no gdp
    return df.drop(countries_no_gdp)

In [ ]:
df_pivoted = df.pivot(index=["CountryCode","Year"], columns="IndicatorCode", values="Value").copy()

In [ ]:
df_cleaned = rm_countries_no_GDP(df_pivoted)

In [ ]:
df_features = add_others_GDP(df_cleaned) 

In [ ]:
df_target = df_pivoted.pop(config.GDP)
df_features.drop(index=2010, level="Year", inplace=True)
df_target.drop(index=1960, level="Year", inplace=True)
# up until here we have the basic features selected. Now we want to add more features, like the GDP of each country


In [ ]:
X = df_features
y = df_target
print(X.shape)
print(y.shape)
imp_mode = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
# genereate imputed dataframes
X_imp = imp_mode.fit_transform(X) 
y_imp = imp_mode.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_imp, y_imp, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
gbm_hyperparams = {
    'n_estimators': 100,
    'max_depth': 10,
    'learning_rate': 0.1,
    'loss': 'ls'
}
gbm_model = GradientBoostingRegressor(**gbm_hyperparams)
gbm_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
y_pred = gbm_model.predict(X_test)
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print(f"RMSE = {rmse} \n R2 = {r2_score(y_test, y_pred)}")


In [ ]:
df_final

In [ ]:
df_final.set_index(df_cleaned.index, inplace=True)

In [ ]:
df_cleaned.join(df_final)